In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [15]:
path = "/app/data/playground-series-s5e12/train.csv"
path_test = "/app/data/playground-series-s5e12/test.csv"

In [16]:
df = pd.read_csv(path)
df_test = pd.read_csv(path_test)

In [17]:
encoding_columns = ['ethnicity', 'education_level', 'income_level', 'smoking_status', 'employment_status', 'gender']

for col in encoding_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

for col in encoding_columns:
    le = LabelEncoder()
    df_test[col] = le.fit_transform(df_test[col]) 

In [18]:
fill_cols = ['family_history_diabetes', 'hypertension_history','cardiovascular_history', 'diagnosed_diabetes']
fill_cols_test = ['family_history_diabetes', 'hypertension_history','cardiovascular_history']


df[fill_cols] = df[fill_cols].fillna(0)
df_test[fill_cols_test] = df_test[fill_cols_test].fillna(0)

In [19]:
df.isna().sum()
df_test.isna().sum()

id                                    0
age                                   0
alcohol_consumption_per_week          0
physical_activity_minutes_per_week    0
diet_score                            0
sleep_hours_per_day                   0
screen_time_hours_per_day             0
bmi                                   0
waist_to_hip_ratio                    0
systolic_bp                           0
diastolic_bp                          0
heart_rate                            0
cholesterol_total                     0
hdl_cholesterol                       0
ldl_cholesterol                       0
triglycerides                         0
gender                                0
ethnicity                             0
education_level                       0
income_level                          0
smoking_status                        0
employment_status                     0
family_history_diabetes               0
hypertension_history                  0
cardiovascular_history                0


In [20]:
df.head(5)
df_test.head(5)

,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,...,triglycerides,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history
0,700000,45,4,100,4.3,6.8,6.2,25.5,0.84,123,...,111,0,4,1,3,1,0,0,0,0
1,700001,35,1,87,3.5,4.6,9.0,28.6,0.88,120,...,145,0,4,1,3,2,3,0,0,0
2,700002,45,1,61,7.6,6.8,7.0,28.5,0.94,112,...,184,1,4,1,1,2,0,0,0,0
3,700003,55,2,81,7.3,7.3,5.0,26.9,0.91,114,...,128,1,4,0,3,1,0,0,0,0
4,700004,77,2,29,7.3,7.6,8.5,22.0,0.83,131,...,133,1,4,0,1,0,3,0,0,0


In [21]:
X = df.drop(columns=['diagnosed_diabetes', 'id'])
Y = df['diagnosed_diabetes']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
model = XGBClassifier(
    n_estimators=1000,
    max_depth=7,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=3,
    random_state=42
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [24]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.6225394302481966


In [25]:
X_test = df_test.drop(columns=["id"])
y_pred = model.predict_proba(X_test)[:, 1]

In [26]:
submission = pd.DataFrame({
    "id": df_test["id"],
    "diagnosed_diabetes": y_pred
})

submission.to_csv('submission.csv', index=False)